# ETL Project
## Irvine Company Apartments


In [1]:
# set environment
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\kate_\.wdm\drivers\chromedriver\win32\88.0.4324.96]


In [3]:
base_url = "https://www.irvinecompanyapartments.com/locations"

In [4]:
browser.visit(base_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [5]:

location_soup = soup.find_all("li", class_= "submarket-listing-item__sub-market-list__item")
for loc in location_soup:
    city_link = loc.find("a", class_="link")["href"]
    print(city_link)

/locations/orange-county/aliso-viejo.html
/locations/orange-county/costa-mesa.html
/locations/orange-county/irvine.html
/locations/orange-county/mission-viejo.html
/locations/orange-county/newport-beach.html
/locations/orange-county/orange.html
/locations/orange-county/rancho-santa-margarita.html
/locations/orange-county/tustin.html
/locations/san-diego/carlsbad.html
/locations/san-diego/carmel-valley.html
/locations/san-diego/downtown.html
/locations/san-diego/la-jolla.html
/locations/san-diego/mission-valley.html
/locations/los-angeles/playa-vista.html
/locations/los-angeles/santa-monica.html
/locations/northern-california/cupertino.html
/locations/northern-california/redwood-city.html
/locations/northern-california/san-jose.html
/locations/northern-california/santa-clara.html
/locations/northern-california/sunnyvale.html


In [6]:
base_url = "https://www.irvinecompanyapartments.com/locations/orange-county/aliso-viejo.html"
browser.visit(base_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [24]:
complex_soup = soup.find_all("div", class_="search-result-item-card__cta-container--bottom")
for loc in complex_soup:
    print(loc.find("a", href=True))

None
<a class="search-result-item-card__cta search-result-item-card__cta--link btn btn-circular-with-icon" href="https://www.irvinecompanyapartments.com/locations/orange-county/aliso-viejo/vista-bella.html" updated-url="https://www.irvinecompanyapartments.com/locations/orange-county/aliso-viejo/vista-bella.html"><span class="apts-icon apts-icon-right-arrow"></span></a>
<a class="search-result-item-card__cta search-result-item-card__cta--link btn btn-circular-with-icon" href="https://www.irvinecompanyapartments.com/locations/orange-county/aliso-viejo/aliso-town-center.html" updated-url="https://www.irvinecompanyapartments.com/locations/orange-county/aliso-viejo/aliso-town-center.html"><span class="apts-icon apts-icon-right-arrow"></span></a>


In [28]:
# build URL for scraping
complex_avail_url = "https://www.irvinecompanyapartments.com/locations/orange-county/irvine/spectrum/promenade-at-spectrum/availability.html#floor-plan-list"

In [29]:
browser.visit(complex_avail_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [30]:
parse_soup = soup
complex_soup = parse_soup.find_all("h1", class_="sticky-header__title-heading")

complex_name = complex_soup[0].text
print(complex_name)

Promenade at Irvine Spectrum


In [31]:

parse_soup = soup
floor_plans = parse_soup.find_all('div', class_="fapt-fp-list-item")
db_list = []
# Loop through returned results
for plan in floor_plans:
    
    avail_dict = {}

    # Error handling
    try:
        avail_dict["plan_name"] = plan.find('div', class_="fapt-fp-list-item__column--plan-name").text
        avail_dict["unit_type"] = plan.find('div', class_="fapt-fp-list-item__column--beds-baths").text
        avail_dict["start_price"] = plan.find('div', class_="fapt-fp-list-item__column--price").text
        avail_dict["sq_ft"] = plan.find('div', class_="fapt-fp-list-item__column--sqft").text
        

        
        units = plan.find_all("div", class_="fapt-fp-unit__table-row")
        
        for unit in units:
            unit_name = unit.find("span", class_="fapt-fp-unit__unit-name-text")

            if unit_name:
                avail_dict["unit_id"] = unit_name.text
                avail_dict["terms"] = unit.find("div", class_="fapt-fp-unit__column-inner--term").text
                avail_dict["curr_price"] = unit.find("div", class_="fapt-fp-unit__column-inner--price").text
                avail_dict["avail_date"] = unit.find("div", class_="fapt-fp-unit__column-inner--available").span.text
              
                db_list.append(avail_dict)
            
    

    except AttributeError as e:
        print(e)
        
print(db_list)

[{'plan_name': 'PLAN 1', 'unit_type': 'Studio / 1 Bath', 'start_price': ' $1,935', 'sq_ft': '681 ', 'unit_id': '05 5114', 'terms': '14 mo.', 'curr_price': '$1,980', 'avail_date': 'Today'}, {'plan_name': 'PLAN 1', 'unit_type': 'Studio / 1 Bath', 'start_price': ' $1,935', 'sq_ft': '681 ', 'unit_id': '05 5114', 'terms': '14 mo.', 'curr_price': '$1,980', 'avail_date': 'Today'}, {'plan_name': 'PLAN 1', 'unit_type': 'Studio / 1 Bath', 'start_price': ' $1,935', 'sq_ft': '681 ', 'unit_id': '05 5114', 'terms': '14 mo.', 'curr_price': '$1,980', 'avail_date': 'Today'}, {'plan_name': 'PLAN 1', 'unit_type': 'Studio / 1 Bath', 'start_price': ' $1,935', 'sq_ft': '681 ', 'unit_id': '05 5114', 'terms': '14 mo.', 'curr_price': '$1,980', 'avail_date': 'Today'}, {'plan_name': 'PLAN 1', 'unit_type': 'Studio / 1 Bath', 'start_price': ' $1,935', 'sq_ft': '681 ', 'unit_id': '05 5114', 'terms': '14 mo.', 'curr_price': '$1,980', 'avail_date': 'Today'}, {'plan_name': 'PLAN 3', 'unit_type': '1 Bed / 1 Bath', 'sta